In [1]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from os import listdir
from os.path import isfile, join
# import matplotlib.pyplot as plt

from pyvis.network import Network # Visualización de los grafos
import networkx as nx
from ipywidgets import interact, interact_manual
import forceatlas2 as fa2
import random

from src.SimilarityArtworks import *
from src.SimilarityUsers import *
from src.ArtworksMatrix import *
from src.UsersMatrix import *
from src.UsersClustering import *
from src.AverageUser import *

In [2]:
import inspect

def inner_classes_list(cls):
    return [cls_attribute.__name__ for cls_attribute in cls.__dict__.values()
            if inspect.isclass(cls_attribute)]
            # and issubclass(cls_attribute, SimilarityFunctionInterface)]

In [3]:
# !pip install cython
# !pip install ForceAtlas2
# import forceatlas2 as fa2

In [4]:
class UserInterface():

    def __init__(self, path='data/'):
      self.path = path
      self.files = [f for f in listdir(self.path) if isfile(join(self.path, f))]
    
### 1. CARGAR DATOS ###
    def loadData(self):
      self.users = pd.read_csv(self.path + self.usersDropdown.value, index_col=0)
      self.items = pd.read_csv(self.path + self.itemsDropdown.value)
      self.users['positive'] = self.users['positive'].apply(eval)
      self.users['negative'] = self.users['negative'].apply(eval)
      self.users['mixed'] = self.users['mixed'].apply(eval)
      self.users_sim_functions = [None]
      self.users_sim_functions.extend(inner_classes_list(SimilarityUsers))
      self.items_sim_functions = [None]
      self.items_sim_functions.extend(inner_classes_list(SimilarityArtworks))
    
    def loadData_create_widgets(self):
      self.completeMessages = widgets.VBox([])
      self.loadDataComplete = widgets.Label(value="- Users and Items data loaded -")
      self.loadDataTitle = widgets.HTML(value="<h2>Select data files</h2>")
      self.usersDropdown = widgets.Dropdown(options=self.files,
                                            value='Prado_users_emotions_OnePolarity.csv',
                                            description='Users:',
                                            disabled=False,
                                            )
      self.itemsDropdown = widgets.Dropdown(options=self.files,
                                            value='Prado_artworks_wikidata.csv',
                                            description='Items:',
                                            disabled=False,
                                            )
      self.loadDataButton = widgets.Button(description="Done", button_style='success')     
      self.loadDataButton.on_click(self.on_loadDataButton_clicked)

    def on_loadDataButton_clicked(self, change):
      with self.out:
        self.out.clear_output()
        self.loadData()
        self.completeMessages.children += (self.loadDataComplete, )
        display(self.completeMessages)
        self.addItemsAtribute_create_widgets()
        display(self.addItemsAtributeTitle)
        display(self.addItemsAtributeButtonBox)
    
### 2. SIMILITUD ITEMS ###
    def computeItemsSim(self):
      self.items_funcs = {x : None for x in self.items.columns}
      self.items_weights = {x : None for x in self.items.columns}  

      for atr in self.items_atribute_list:
          if atr.children[0].value not in self.items.columns: # Atributo con fichero externo
              self.items_funcs.update({self.path + atr.children[0].value : atr.children[1].value})
              self.items_weights.update({self.path + atr.children[0].value : atr.children[2].value})
          else:
              self.items_funcs.update({atr.children[0].value : atr.children[1].value})
              self.items_weights.update({atr.children[0].value : atr.children[2].value})
      AM = ArtworksMatrix(self.items, function_dict=self.items_funcs, weight_dict=self.items_weights)#, colors_path=self.path + self.addItemsAtributeColor.children[0].value)
      self.itemsMatrix = AM.getSimilarityMatrix()

    def getItemsSim(self):
      self.items_funcs = {}
      self.itemsMatrix = pd.read_csv(self.path + self.itemsSimilarityFileDropdown.value, index_col = 0)
        
    def addItemsAtribute_create_widgets(self):
      self.items_atribute_list = []
      self.default_items_funcs = []
      self.addItemsAtributeComplete = widgets.Label(value="- Items similarity computed -")
      self.addItemsAtributeTitle = widgets.HTML(value="<h2>Add atributes to compute ITEMS similarity</h2>")
      self.addItemsAtributeFileTitle = widgets.HTML(value="<h2>Select a file with the ITEMS similarity matrix</h2>")
      
      self.addItemsAtributeButton = widgets.Button(description="Atribute", icon='plus')
      self.addItemsAtributeButton.on_click(self.on_addItemsAtributeButton_clicked) 
      self.addItemsAtributeButtonExtra = widgets.Button(description="File Atribute", icon='plus')
      self.addItemsAtributeButtonExtra.on_click(self.on_addItemsAtributeButtonExtra_clicked)
      self.addItemsAtributeButtonFile = widgets.Button(description="Similarity File", icon='plus', button_style='info')
      self.addItemsAtributeButtonFile.on_click(self.on_addItemsAtributeButtonFile_clicked)        
      self.addItemsAtributeButtonBox = widgets.HBox([self.addItemsAtributeButton, self.addItemsAtributeButtonExtra,self.addItemsAtributeButtonFile])
      self.loadItemsAtributeButton = widgets.Button(description="Compute similarity", button_style='success')
      self.loadItemsAtributeButton.on_click(self.on_loadItemsAtributeButton_clicked)
    
      self.simItemsFileButton = widgets.Button(description="File selected", button_style='success',)
      self.simItemsFileButton.on_click(self.on_simItemsFileButton_clicked)
    
    def addItemsAtribute(self):
      aux1 = widgets.Dropdown(options=self.items.columns, value=None, description='Atribute:',disabled=False)
      aux2 = widgets.Dropdown(options=self.items_sim_functions, description='Sim Function:', disabled=False)
      # aux3 = widgets.BoundedFloatText(value=0.0, min=0, max=1, step=0.1, description='Weight:', disabled=False)
      aux3 = widgets.FloatSlider(value=0.0, min=0, max=1, step=0.1, description='Weight:', disabled=False, readout=True, readout_format='.1f')
      
      self.itemsLastAtribute = widgets.HBox([aux1, aux2, aux3])
    
    def addItemsAtributeExtra(self):
      aux1 = widgets.Dropdown(options=self.files, value=None, description='File:', disabled=False)
      aux2 = widgets.Dropdown(options=self.items_sim_functions, value=None, description='Sim Function:', disabled=False)
      # aux3 = widgets.BoundedFloatText(value=0.0, min=0, max=1, step=0.1, description='Weight:', disabled=False)
      aux3 = widgets.FloatSlider(value=0.0, min=0, max=1, step=0.1, description='Weight:', disabled=False, readout=True, readout_format='.1f')
     
      self.itemsLastAtribute = widgets.HBox([aux1, aux2, aux3])
    
    def addItemsSimilarityFile(self):
        self.itemsSimilarityFileDropdown = widgets.Dropdown(options=self.files, value=None, description='Sim file:', disabled=False)
        
    def on_addItemsAtributeButton_clicked(self, change):
      with self.out:
        self.out.clear_output()
        self.addItemsAtribute()
        display(self.completeMessages)
        display(self.addItemsAtributeTitle)
        for atr in self.items_atribute_list:
          display(atr)
        display(self.itemsLastAtribute)
        display(self.addItemsAtributeButtonBox)
        display(self.loadItemsAtributeButton)
        self.items_atribute_list.append(self.itemsLastAtribute)
    
    def on_addItemsAtributeButtonExtra_clicked(self, change):
      with self.out:
        self.out.clear_output()
        self.addItemsAtributeExtra()
        display(self.completeMessages)
        display(self.addItemsAtributeTitle)
        for atr in self.items_atribute_list:
          display(atr)
        display(self.itemsLastAtribute)
        display(self.addItemsAtributeButtonBox)
        display(self.loadItemsAtributeButton)
        self.items_atribute_list.append(self.itemsLastAtribute)    
        
    def on_addItemsAtributeButtonFile_clicked(self, change):
      with self.out:
        self.out.clear_output()
        self.addItemsSimilarityFile()
        display(self.completeMessages)
        display(self.addItemsAtributeFileTitle)
        display(self.itemsSimilarityFileDropdown)
        display(self.simItemsFileButton)
        # self.items_atribute_list.append(self.itemsLastAtribute)   
    
    def on_loadItemsAtributeButton_clicked(self, change):
      with self.out:
        self.out.clear_output()
        display(self.completeMessages)
        print('Computing items similarity...')
        self.computeItemsSim()
        self.out.clear_output()
        self.completeMessages.children += (self.addItemsAtributeComplete, )
        display(self.completeMessages)
        self.addUsersAtribute_create_widgets()
        display(self.addUsersAtributeTitle)
        # display(self.addUsersAtributeButton)
        display(self.addUsersAtributeButtonBox)    
    def on_simItemsFileButton_clicked(self, change):
        with self.out:
            self.out.clear_output()
            display(self.completeMessages)
            print('Getting items similarity file...')
            self.getItemsSim()
            self.out.clear_output()
            self.completeMessages.children += (self.addItemsAtributeComplete, )
            display(self.completeMessages)
            self.addUsersAtribute_create_widgets()
            display(self.addUsersAtributeTitle)
            for atr in self.users_atribute_list:
                display(atr)
            # display(self.addUsersAtributeButton)     
            display(self.addUsersAtributeButtonBox)
        
    
### 3. SIMILITUD USUARIOS ###
    def computeUsersSim(self):
      self.users_funcs = {x : None for x in self.users.columns}
      self.users_weights = {x : None for x in self.users.columns}  
      self.users_atribute_list.remove(self.demogPolBox)
      for atr in self.users_atribute_list:
        self.users_funcs.update({atr.children[0].value : atr.children[1].value})
        self.users_weights.update({atr.children[0].value : atr.children[2].value})
      
      self.users_weights.update({'polarity' : self.polWeight, 'demographic' : self.demogWeight}) ######
      
      UM = UsersMatrix(self.users, self.itemsMatrix, function_dict=self.users_funcs, weight_dict=self.users_weights)
      self.usersMatrix = UM.getSimilarityMatrix()
    
    def getUsersSim(self):
      self.users_funcs = {}
      self.usersMatrix = pd.read_csv(self.path + self.usersSimilarityFileDropdown.value,index_col=0)   

    
    def addUsersAtribute_create_widgets(self):
      self.loadUsersAtributeSlider = widgets.FloatSlider(value=0.0, min=0, max=1, step=0.1, disabled=False, readout=False)
      self.demogLabel = widgets.HTML(value='<font size=\"+0.5\">Demographic weight <b>1.0</b></font>')
      self.polLabel = widgets.HTML(value='<font size=\"+0.5\"><b>0.0</b> Polarity weight</b></font>')
      self.demogPolBox = widgets.HBox([self.demogLabel, self.loadUsersAtributeSlider, self.polLabel])
      self.demogWeight = 1.0
      self.polWeight = 0.0

      def link_sliders(change):
        self.demogWeight = 1 - change.new
        self.polWeight = change.new
        self.demogLabel.value = "<font size=\"+0.5\">Demographic weight <b>{:.1f}</b></font>".format(self.demogWeight)
        self.polLabel.value = "<font size=\"+0.5\"><b>{:.1f}</b> Polarity weight</font>".format(self.polWeight)
        
      self.loadUsersAtributeSlider.observe(link_sliders, names='value')
      
      self.users_atribute_list = [self.demogPolBox]
      self.default_users_funcs = []
      
      self.addUsersAtributeComplete = widgets.Label(value="- Users similarity computed -")
      self.addUsersAtributeTitle = widgets.HTML(value="<h2>Add atributes to compute USERS similarity</h2>")
      self.addUsersAtributeFileTitle = widgets.HTML(value="<h2>Select a file with the USERS similarity matrix</h2>")
      
      self.addUsersAtributeButton = widgets.Button(description="Atribute", icon='plus')
      self.addUsersAtributeButton.on_click(self.on_addUsersAtributeButton_clicked) 
      self.addUsersAtributeButtonFile = widgets.Button(description="Similarity File", icon='plus', button_style='info')
      self.addUsersAtributeButtonFile.on_click(self.on_addUsersAtributeButtonFile_clicked)        
      self.addUsersAtributeButtonBox = widgets.HBox([self.addUsersAtributeButton, self.addUsersAtributeButtonFile])
      
      self.loadUsersAtributeButton = widgets.Button(description="Compute similarity", button_style='success')
      self.loadUsersAtributeButton.on_click(self.on_loadUsersAtributeButton_clicked)
      
      self.loadUsersAtributeFileButton = widgets.Button(description="File selected", button_style='success',)
      self.loadUsersAtributeFileButton.on_click(self.on_simUsersFileButton_clicked)  
    
    def addUsersAtribute(self):
      aux1 = widgets.Dropdown(options=self.users.columns, value=None, description='Atribute:', disabled=False)
      aux2 = widgets.Dropdown(options=self.users_sim_functions, description='Sim Function::', disabled=False)
      # aux3 = widgets.BoundedFloatText(value=0.0, min=0, max=1, step=0.1, description='Weight:', disabled=False)
      aux3 = widgets.FloatSlider(value=0.0, min=0, max=1, step=0.1, description='Weight:', disabled=False, readout=True, readout_format='.1f')
      
      self.usersLastAtribute = widgets.HBox([aux1, aux2, aux3])
    
    def addUsersSimilarityFile(self):
        self.usersSimilarityFileDropdown = widgets.Dropdown(options=self.files, value=None, description='Sim file:', disabled=False)
    
    def on_addUsersAtributeButton_clicked(self, change):
      self.out.clear_output()
      self.addUsersAtribute()
      with self.out:
        display(self.completeMessages)
        display(self.addUsersAtributeTitle)
        for atr in self.users_atribute_list:
          display(atr)
        display(self.usersLastAtribute)
        # display(self.addUsersAtributeButton)
        display(self.addUsersAtributeButtonBox)
        display(self.loadUsersAtributeButton)
        self.users_atribute_list.append(self.usersLastAtribute)

    def on_addUsersAtributeButtonFile_clicked(self, change):
      with self.out:
        self.out.clear_output()
        self.addUsersSimilarityFile()
        display(self.completeMessages)
        display(self.addUsersAtributeFileTitle)
        display(self.usersSimilarityFileDropdown)
        display(self.loadUsersAtributeFileButton)
        # self.users_atribute_list.append(self.usersLastAtribute)
    
    def on_loadUsersAtributeButton_clicked(self, change):
      with self.out:
        self.out.clear_output()
        display(self.completeMessages)
        print('Computing users similarity...')
        self.computeUsersSim()
        self.out.clear_output()
        self.completeMessages.children += (self.addUsersAtributeComplete, )
        display(self.completeMessages)
        self.computeClusters_create_widgets()
        display(self.computeClustersTitle)
        display(self.clusterFunction)
        display(self.computeClustersButton)

    def on_simUsersFileButton_clicked(self, change):
      with self.out:
        self.out.clear_output()
        display(self.completeMessages)
        print('Getting users similarity file...')
        self.getUsersSim()

        self.out.clear_output()
        self.completeMessages.children += (self.addUsersAtributeComplete, )
        display(self.completeMessages)
        self.computeClusters_create_widgets()
        display(self.computeClustersTitle)
        display(self.clusterFunction)
        display(self.computeClustersButton)
      
    
### 4. CLUSTERING ###
    def computeClusters(self):
      users_distances = 1 - self.usersMatrix
      self.UC = UsersClustering(users_distances)
      funCluster = getattr(self.UC, self.clusterFunction.value)
      labels = funCluster()
      # labels = UsersClustering(users_distances).dbscanFromMatrix()
      self.users_clustered = self.users.copy()
      self.users_clustered['cluster'] = labels
      
    def computeClusters_create_widgets(self):
      self.computeClustersComplete = widgets.Label(value="- Users clustered -")
      self.computeClustersTitle = widgets.HTML(value="<h2>Compute users clusters</h2>")
      self.computeClustersButton = widgets.Button(description="Compute clusters", button_style='success')
      self.computeClustersButton.on_click(self.on_computeClustersButton_clicked)  
      self.clusterFunction = widgets.Dropdown(options=[x[0] for x in inspect.getmembers(UsersClustering, predicate=inspect.isfunction)[1:] if (x[0] != 'daviesBouldinScore')], value='kMedoidsFromMatrix', description='Alg Cluster:',disabled=False)
    
    def on_computeClustersButton_clicked(self, change):
      with self.out:
        self.out.clear_output()
        display(self.completeMessages)
        print('Computing users clusters...')
        self.computeClusters()
        self.out.clear_output()
        self.completeMessages.children += (self.computeClustersComplete, )
        display(self.completeMessages)
        self.showExplanation_create_widgets()
        display(self.showExplanationTitle)
        print("ITEM ATRIBUTES")
        display(self.addDefaultItemsCheckboxBox)
        print("USERS ATRIBUTES")
        display(self.addDefaultUsersCheckboxBox)
        display(self.showExplanationButton)
    
### 5. EXPLICACION ###
    def showExplanation(self):  
      self.users_atributes = self.default_users_funcs#[key for key, value in self.users_funcs.items() if value is not None] if len(self.default_users_funcs)==0 else self.default_users_funcs
      self.items_atributes = self.default_items_funcs#[key for key, value in self.items_funcs.items() if value is not None] if len(self.default_items_funcs)==0 else self.default_items_funcs
      AU = AverageUser(self.users_clustered, self.items, self.users_atributes, self.items_atributes)
      n_artworks = 3
      self.explicators = AU.computeAverageUser(n_artworks)
      AU.computeInfographics()
      self.expl = AU.returnExplanation()
      self.clustersTab.children = [widgets.HTML(value=str(self.expl[i]) +  "<img src=\" https://uploads7.wikiart.org/images/francisco-goya/charles-iv-of-spain-and-his-family-1800.jpg!Large.jpg\">") for i in self.expl.keys()]
      for i in range(len(self.clustersTab.children)):
        self.clustersTab.set_title(i,"cluster " + str(i))
      self.expl_json = AU.returnJSONExplanation()

    
    def explanation(self):
        path_img = "cache/"
        info = json.loads(self.expl_json)
        children_cluster = [widgets.Tab() for i in info]
        c = 0
        for cluster in info:
            # demograficos
            demog_box = widgets.VBox()
            children_demog_box = [widgets.HBox() for atr in info[cluster]['usr']]
            i = 0
            for atr in info[cluster]['usr']:
                if atr == "Individuos":
                    demog_text_indiv = widgets.HTML(value = "<b>" + atr + "</b>: " + str(info[cluster]['usr'].get(atr)) + "<br>")
                    children_demog_box[i] = demog_text_indiv
                else:
                    demog_box_atr = widgets.HBox()
                    demog_text_atr = widgets.HTML()
                    demog_text_atr.value += "<b>" + atr + "</b>:<br>"
                    for atr_mode in info[cluster]['usr'][atr]:
                        demog_text_atr.value += "&emsp;" + atr_mode + " con un porcentaje de " + str(info[cluster]['usr'][atr].get(atr_mode)) + "</br>"
                    img_name = path_img + str(cluster) + "_" + atr + ".png"
                    file = open(img_name, "rb")
                    image = file.read()
                    demog_img_atr =  widgets.Image(value = image)
                    demog_box_atr.children = [demog_text_atr,demog_img_atr]
                    children_demog_box[i] = demog_box_atr
                    
                i+=1
            demog_box.children = children_demog_box            
            # gustos
            children_pol = [widgets.Accordion() for i in range(len(np.intersect1d(self.users_atributes,['positive', 'negative', 'mixed'])))]
            pol = 0
            for polarity in (np.intersect1d(self.users_atributes,['positive', 'negative', 'mixed'])):
                children_artworks = [widgets.HTML() for i in range(len(info[cluster]['polarity'][polarity]))]
                for artwork in range(len(info[cluster]['polarity'][polarity])):
                    for atr in info[cluster]['polarity'][polarity][artwork]:
                        if atr == "Image URL" :
                            children_artworks[artwork].value += "<p><img src=\"" + info[cluster]['polarity'][polarity][artwork].get(atr) + "\"></p>" 
                        elif atr != "title" and atr != "Title":
                            children_artworks[artwork].value += "<p><b>" + atr + "</b>:" + info[cluster]['polarity'][polarity][artwork].get(atr) + "</p>"
                acc = widgets.Accordion(children = children_artworks)
                for i in range(len(children_artworks)):
                    acc.set_title(i, info[cluster]['polarity'][polarity][i].get('title'))
                children_pol[pol] = acc
                pol += 1
            tabPol = widgets.Tab(children = children_pol)
            for i in range(len(children_pol)):
                tabPol.set_title(i, np.intersect1d(self.users_atributes,['positive', 'negative', 'mixed'])[i])
                
            # if the user only chooses demographic attributes to show
            if (len(info[cluster]["polarity"]) == 0):
                children_cluster[c] = demog_box
            else:
                children_cluster[c] = widgets.VBox([demog_box,tabPol])
            c +=1
        tabClusters = widgets.Tab(children = children_cluster)
        index = 0
        for j in info:
            tabClusters.set_title(index,"Cluster " + str(j))
            index+=1
        display(tabClusters)
                
            
      
    def showExplanation_create_widgets(self):
      self.clustersTab = widgets.Tab()
      self.showExplanationTitle = widgets.HTML(value="<h2>Clusters explanation</h2>")
      self.showExplanationButton = widgets.Button(description="Show explanation", button_style='success')
      self.showExplanationButton.on_click(self.on_showExplanationButton_clicked)      
      
      self.addDefaultItemsCheckbox = [widgets.Checkbox(description=atr, value=False, width='10px') for atr in self.items.columns]
      for i in self.addDefaultItemsCheckbox: 
        i.style.description_width = 'initial'
        if self.items_funcs.get(i.description) is not None: 
          i.value = True
        
      self.addDefaultItemsCheckboxBox = widgets.HBox(children = self.addDefaultItemsCheckbox, layout=widgets.Layout(width="800px"))
      
      self.addDefaultUsersCheckbox = [widgets.Checkbox(description=atr, width=1) for atr in self.users.columns]
      for i in self.addDefaultUsersCheckbox: 
        i.style.description_width = 'initial'
        if self.users_funcs.get(i.description) is not None: 
          i.value = True
      self.addDefaultUsersCheckboxBox = widgets.HBox(children = self.addDefaultUsersCheckbox, layout=widgets.Layout(width="800px"))
      
      # self.loadShowAtributesButton = widgets.Button(description="Atributes selected", button_style='success')
      # self.loadShowAtributesButton.on_click(self.on_loadShowAtributesButton_clicked)
      
    # def on_loadShowAtributesButton_clicked(self,change):
    #   with self.out:
    #     self.out.clear_output()
    #     self.default_items_funcs = [x.description for x in self.addDefaultItemsCheckbox if x.value == True]
    #     self.default_users_funcs = [x.description for x in self.addDefaultUsersCheckbox if x.value == True]
    #     self.showExplanation()
    #     display(self.completeMessages)
    #     self.explanation()
    #     self.showGraph_create_widgets()
    #     display(self.showGraphTitle)
    #     display(self.showGraphButton)

    def on_showExplanationButton_clicked(self, change):
      with self.out:
        self.out.clear_output()
        self.default_items_funcs = [x.description for x in self.addDefaultItemsCheckbox if x.value == True]
        self.default_users_funcs = [x.description for x in self.addDefaultUsersCheckbox if x.value == True]
        self.showExplanation()
        display(self.completeMessages)
        self.explanation()
        # display(self.clustersTab)
        self.showGraph_create_widgets()
        display(self.showGraphTitle)
        display(self.showGraphButton)

        
### 6. GRAFO ###
    def createGraph(self):
      self.net = Network(notebook=True, width='100%', height='800px')
      self.colors = ["blue","orange","green","red","purple","brown","pink","gray", "olive", "cyan", "black", "b", "g", "r", "c", "m","y"]    
      
      self.G = nx.Graph()
      
      ## FALTA AÑADIR EXPLICADORES
      for i in range(len(self.users_clustered)):
        usr = self.users_clustered.loc[i]
        usr['positive'] = [self.items.loc[self.items['ID'] == id, 'Title'].item() for id in usr['positive']]
        usr['negative'] =  [self.items.loc[self.items['ID'] == id,'Title'].item() for id in usr['negative']]
        usr['mixed'] = [self.items.loc[self.items['ID'] == id,'Title'].item() for id in usr['mixed']]
        title = '<p>' + ''.join('{bold}'.format(bold = '<b>' if self.users_funcs.get(self.users_clustered.columns[e]) is not None else '')
                                + str(self.users_clustered.columns[e]) 
                                + '{endBold} :'.format(endBold = '</b>' if self.users_funcs.get(self.users_clustered.columns[e]) is not None else '') 
                                + str(usr[e]) 
                                + "<br>" for e in range(len(usr.to_list()))) +'</p>'
        self.net.add_node(n_id=int(usr.userId),
                     label=str(usr.userId),
                     title= title,
                     color=self.colors[usr.cluster])
        self.G.add_nodes_from([(int(usr.userId), {'pos' : 0, 'color' : self.colors[usr.cluster]})])
      
      for i in range(len(self.usersMatrix.values)):
        for j in range(len(self.usersMatrix.values[i])):
          # if i != j and self.usersMatrix.values[i][j] > 0.5:
          if i != j:
            a = self.net.nodes[i].get('id')
            b = self.net.nodes[j].get('id')
            self.net.add_edge(a, b, weight=self.usersMatrix.values[i][j], hidden=True)
            self.G.add_edges_from([(a, b, {'weight' : self.usersMatrix.values[i][j]})])
            
          # if self.usersMatrix.values[i][j] > 0:
          # self.net.add_edge(a, b, weight=self.usersMatrix.values[i][j]*self.usersMatrix.values[i][j], hidden=True)
      
      pos = { i : (random.random(), random.random()) for i in self.G.nodes()} # Optionally specify positions as a dictionary 
      l = fa2.forceatlas2_networkx_layout(self.G, pos, niter=100, edgeWeightInfluence=3, scalingRatio=5.0, gravity=100)
      x = { i : (random.random(), random.random()) for i in UI.G.nodes()}
      y = { i : (random.random(), random.random()) for i in UI.G.nodes()}
      for k, v in l.items():
        x.update({k : v[0]})
        y.update({k : v[1]})
      nx.set_node_attributes(self.G, x , name='x')
      nx.set_node_attributes(self.G, y , name='y')
      
      self.net = Network(notebook=True, width='100%', height='800px')
      self.net.from_nx(self.G)
      for edge in self.net.edges:
        edge.update({'hidden' : True})

      for node in self.net.nodes:
        node.pop('label', None)
        node.update({'title' : 'A'})
    
    def showGraph(self):
      self.net.toggle_physics(False)
      self.net.width = '75%'
      # self.net.show_buttons(filter_=['physics'])
      return self.net.show('nodes.html')

    def showGraph_create_widgets(self):
      self.showGraphTitle = widgets.HTML(value="<h2>Users graph</h2>")
      self.showGraphButton = widgets.Button(description="Show graph", button_style='success')
      self.showGraphButton.on_click(self.on_showGraphButton_clicked)  

    def on_showGraphButton_clicked(self, change):
      with self.out:
        # self.out.clear_output()
        # display(self.completeMessages)
        self.createGraph()
        display(self.showGraph())
    
    
### MOSTAR TODO ###
    def display_widgets(self):
        self.loadData_create_widgets()
        self.out = widgets.Output()  # this is the output widget in which the df is displayed
        
        display(self.out)
        with self.out:
          display(self.loadDataTitle)
          display(widgets.VBox(
                              [
                                  self.usersDropdown,
                                  self.itemsDropdown,
                                  self.loadDataButton,
                              ]
                          )
                 )


## RUN ##
UI = UserInterface()
UI.display_widgets()

Output()

In [55]:
from src.gephiVisualization import GephiVisualization
users = UI.users_clustered[['userId', 'age', 'gender', 'country', 'cluster']]
GephiVisualization().load_community(users=users.values, distances=UI.usersMatrix.values, users_properties=users.values[:, 1:].reshape(-1))

In [296]:
import random
G_force = UI.G.copy()

In [302]:
# !pip install cython
# !pip install ForceAtlas2

pos = { i : (random.random(), random.random()) for i in UI.G.nodes()} # Optionally specify positions as a dictionary 
l = fa2.forceatlas2_networkx_layout(G_force, pos, niter=100, edgeWeightInfluence=3, scalingRatio=5.0, gravity=100)  # Optionally specify iteration count 

In [303]:
x = { i : (random.random(), random.random()) for i in UI.G.nodes()}
y = { i : (random.random(), random.random()) for i in UI.G.nodes()}
for k, v in l.items():
  x.update({k : v[0]})
  y.update({k : v[1]})

In [297]:
pos = nx.spring_layout(G_force, scale=300)

In [298]:
x = { i : (random.random(), random.random()) for i in G_force.nodes()}
y = { i : (random.random(), random.random()) for i in G_force.nodes()}

for k, v in pos.items():
  x.update({k : v[0]})
  y.update({k : v[1]})

In [304]:
nx.set_node_attributes(G_force, x , name='x')
nx.set_node_attributes(G_force, y , name='y')

In [312]:
nt = Network(notebook=True)
nt.from_nx(G_force)
for edge in nt.edges:
  edge.update({'hidden' : True})
  
for node in nt.nodes:
  node.pop('label', None)
  node.update({'title' : 'A'})

In [313]:
nt.toggle_physics(False)
nt.show_buttons(filter_=['nodes'])
# nt.show_buttons()
nt.show('nt.html')